### Load MIMIC data

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import numpy as np

# Load data
file_path = r"E:\data.csv"
data = pd.read_csv(file_path)
data.head()

### Data preprocessing

In [ ]:
# Print number of samples before preprocessing
print("Number of samples before preprocessing:", len(data))

# Data preprocessing
# Filter: Age >= 18
filtered_data1 = data[data['age'] >= 18]
print("Number of samples after filtering (Age >= 18):", len(filtered_data1))

# Filter: Exclude myocardial infarction patients
filtered_data2 = filtered_data1[filtered_data1['myocardial_infarct'] != 1]
print("Number of samples after filtering (Non-myocardial infarction):", len(filtered_data2))

# Filter: Exclude congestive heart failure patients
filtered_data3 = filtered_data2[filtered_data2['congestive_heart_failure'] != 1]
print("Number of samples after filtering (Non-congestive heart failure):", len(filtered_data3))

# Filter: ICU length of stay >= 0.125 days (3 hours)
filtered_data4 = filtered_data3[filtered_data3['los_icu'] >= 0.125]
print("Number of samples after filtering (ICU LOS >= 3 hours):", len(filtered_data4))

# Final dataset
filtered_data = filtered_data4

# Print final label distribution
label_distribution = filtered_data['label'].value_counts()
print("\nLabel distribution in the final dataset:")
print(label_distribution)


### Delete missing values

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

def process_race(df, race_column='race'):
    white_keywords = ['WHITE', 'CAUCASIAN', 'PORTUGUESE', 'EUROPEAN', 'RUSSIAN', 'BRAZILIAN']
    black_keywords = ['BLACK', 'AFRICAN']
    asian_keywords = ['ASIAN', 'CHINESE', 'KOREAN', 'INDIAN']
    hispanic_keywords = ['HISPANIC', 'LATINO', 'SALVADORAN', 'GUATEMALAN', 'HONDURAN', 
                        'MEXICAN', 'DOMINICAN', 'COLUMBIAN', 'CUBAN', 'PUERTO RICAN',
                        'SOUTH AMERICAN', 'CENTRAL AMERICAN']
    
    def categorize_race(x):
        if pd.isna(x):
            return 'Other'
        x = str(x).upper()
        if any(keyword in x for keyword in white_keywords):
            return 'White'
        elif any(keyword in x for keyword in black_keywords):
            return 'Black'
        elif any(keyword in x for keyword in asian_keywords):
            return 'Asian'
        elif any(keyword in x for keyword in hispanic_keywords):
            return 'Hispanic'
        else:
            return 'Other'
    
    return df[race_column].apply(categorize_race)

# Separate features and labels
features = filtered_data[['gender', 'age', 'race',
'd_dimer', 'fibrinogen', 'thrombin', 'inr', 'pt', 'ptt', 
'albumin', 'globulin', 'chloride', 'aniongap', 'bicarbonate', 
'calcium', 'total_protein', 'bun', 'potassium', 'sodium', 'glucose', 'creatinine',
'alt','alp','ast','bilirubin_indirect','ggt','ck_mb','ck_cpk','bilirubin_direct','amylase','ld_ldh','bilirubin_total',
'monocytes', 'nrbc','metamyelocytes','immature_granulocytes','bands','atypical_lymphocytes','neutrophils','lymphocytes',
'basophils','neutrophils_abs','monocytes_abs','lymphocytes_abs','eosinophils_abs','basophils_abs','wbc','eosinophils',
'mchc','rdw','rbc','platelet','mcv','rdwsd','mch','hemoglobin','hct', 'so2', 'glucose','lactate','potassium','temperature',
'calcium','chloride','carboxyhemoglobin','hemoglobin','hct','methemoglobin','bicarbonate','po2','totalco2',
'fio2_chartevents','fio2','aado2','pco2','pao2fio2ratio','ph','baseexcess','aado2_calc','a','a_unit','b','b_unit','c','c_unit','d','d_unit','e','e_unit','f','f_unit','g','g_unit','h','h_unit',
'weight','height','hr','sbp','sbp_ni','dbp','dbp_ni','mbp','mbp_ni','rr','temperature','spo2','glucose',
'age_score','peripheral_vascular_disease','cerebrovascular_disease','dementia','chronic_pulmonary_disease','rheumatic_disease',
'peptic_ulcer_disease','mild_liver_disease','diabetes','paraplegia','renal_disease','severe_liver_disease',
'metastatic_solid_tumor','aids','gcs']]
labels = filtered_data['label']


# Processing racial data
features['race'] = process_race(features)
print(features['race'].value_counts())
print("="*50)

# Delete samples with null gender
features = features.dropna(subset=['gender'])
labels = labels[features.index]

# Save original index
original_index = features.index

# Print initial data information
print("Initial data information：")
print(f"Number of features: {features.shape[1]}")
print(f"Number of samples: {features.shape[0]}")
print("="*50)

# Count the number and proportion of missing values in each column
missing_counts_before = features.isna().sum()
missing_ratio_before = (missing_counts_before / len(features) * 100)
missing_ratio_sorted = missing_ratio_before[missing_ratio_before > 0].sort_values(ascending=False)

# Draw distribution map of missing values
plt.figure(figsize=(10, 12))
bars = plt.barh(missing_ratio_sorted.index, missing_ratio_sorted, 
                color='skyblue', edgecolor='black')

# Add proportion label
for bar, ratio in zip(bars, missing_ratio_sorted):
    plt.text(ratio + 0.5, bar.get_y() + bar.get_height()/2, 
             f"{ratio:.1f}%", va='center', fontsize=9)

plt.title("Missing value distribution (before processing)", fontsize=14)
plt.xlabel("Percentage of missing values (%)", fontsize=12)
plt.ylabel("Characteristic", fontsize=12)
plt.tight_layout()
plt.show()

# Delete high missing rate columns (threshold 30%)
threshold = 0.3 * len(features)
columns_to_drop = features.columns[features.isna().sum() > threshold]
features = features.drop(columns=columns_to_drop)

print(f"Delete columns with a missing rate exceeding 30%, removing {len (columns_to_drop)} features in total:")
print(columns_to_drop.tolist())
print("="*50)

# Standardize numerical features
numeric_features = features.select_dtypes(include=[np.number]).columns
scalers = {}
for feature in numeric_features:
    scalers[feature] = StandardScaler()
    features.loc[:, feature] = scalers[feature].fit_transform(features[[feature]].fillna(0))

# Encode categorical variables
# gender coding
if 'gender' in features.columns:
    gender_mapping = {
        'F': 0, 'f': 0, 
        'M': 1, 'm': 1,
        '0': 0, '0.0': 0, 0: 0, 0.0: 0,
        '1': 1, '1.0': 1, 1: 1, 1.0: 1
    }

    print(features['gender'].unique())
    features['gender'] = features['gender'].map(gender_mapping)
    
    print(features['gender'].value_counts())
    print("="*50)

# racial coding
if 'race' in features.columns:
    le = LabelEncoder()
    features['race'] = le.fit_transform(features['race'])
    for i, label in enumerate(le.classes_):
        print(f"{label}: {i}")
    print("="*50)

# Using KNNImputer to fill in missing values
imputer = KNNImputer(n_neighbors=5, weights='distance')
features_imputed_array = imputer.fit_transform(features)

features_imputed = pd.DataFrame(features_imputed_array, 
                              columns=features.columns,
                              index=original_index)

for feature in numeric_features:
    if feature in features_imputed.columns:
        features_imputed.loc[:, feature] = scalers[feature].inverse_transform(features_imputed[[feature]])

remaining_missing = features_imputed.isnull().sum()
if remaining_missing.sum() > 0:
    print("Warning: Columns with missing values still exist after filling：")
    print(remaining_missing[remaining_missing > 0])
else:
    print("All missing values have been successfully filled in")

features = features_imputed

assert len(features) == len(labels), "Feature and label length do not match"
assert (features.index == labels.index).all(), "The indexes of features and labels do not match"

print("="*50)
print("Final data processing result：")
print(f"Retain the number of features: {features.shape[1]}")
print(f"Retain sample quantity: {features.shape[0]}")
print("="*50)

final_features = features.columns.tolist()
print("\nReserved feature list：")
print(final_features)

# Basic statistical information
print("\nBasic statistical information of features：")
print(features.describe().round(2))

### Downsampling processing

In [ ]:
# Print class distribution before downsampling
print("\nClass distribution before downsampling:")
print(labels.value_counts())

# Get data for each class
df_0 = features[labels == 0]
df_1 = features[labels == 1]

# Get the number of samples in the minority class
num_samples = len(df_1)
print(f"\nNumber of minority class samples: {num_samples}")

# Downsample the majority class
df_0_downsampled = df_0.sample(n=num_samples, random_state=42)

# Combine the downsampled majority class with the minority class
features_balanced = pd.concat([df_0_downsampled, df_1])
labels_balanced = pd.concat([pd.Series(0, index=df_0_downsampled.index),
                             pd.Series(1, index=df_1.index)])

# Shuffle the combined data
idx = np.random.permutation(len(features_balanced))
features = features_balanced.iloc[idx].reset_index(drop=True)
labels = labels_balanced.iloc[idx].reset_index(drop=True)

# Print class distribution after downsampling
print("\nClass distribution after downsampling:")
print(labels.value_counts())

### Normalize the specified features

In [ ]:
# Normalize continuous features
features_to_normalize = ['age', 'inr', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc',
                         'mchc', 'rdw', 'rbc', 'platelet', 'mcv', 'mch', 'weight']

# Extract features to be normalized
features_to_normalize_data = features[features_to_normalize]

# Save original features for reference
original_features = features.copy()

# Perform normalization
scaler = StandardScaler()
features_to_normalize_scaled = scaler.fit_transform(features_to_normalize_data)

# Convert normalized data to DataFrame
features_to_normalize_scaled_df = pd.DataFrame(features_to_normalize_scaled,
                                                columns=features_to_normalize,
                                                index=features.index)

# Update the features DataFrame with normalized values
features.update(features_to_normalize_scaled_df)

# Check missing values after normalization
missing_values_after_normalization = features.isna().sum()
print("\nMissing value statistics after normalization:")
print(missing_values_after_normalization[missing_values_after_normalization > 0])

print("\nFeatures data after normalization:")
print(features.head())


### Boruta feature selection

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from boruta import BorutaPy
import numpy as np
import random


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)



def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    train_sens = recall_score(y_train, y_train_pred)
    test_sens = recall_score(y_test, y_test_pred)
    
    train_spec = recall_score(y_train, y_train_pred, pos_label=0)
    test_spec = recall_score(y_test, y_test_pred, pos_label=0)
    
    train_auroc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    test_auroc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    train_ppv = precision_score(y_train, y_train_pred)
    test_ppv = precision_score(y_test, y_test_pred)
    
    train_npv = precision_score(y_train, y_train_pred, pos_label=0)
    test_npv = precision_score(y_test, y_test_pred, pos_label=0)

    train_f1 = f1_score(y_train, y_train_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_test_proba)
    roc_auc = auc(fpr, tpr)
    
    
    return train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model


def boruta_feature_selection(X, y, feature_names):

    # Create a copy of the data
    X_processed = X.copy()
    
    # Process categorical variables
    for column in X_processed.columns:
        if X_processed[column].dtype == 'object' or X_processed[column].dtype == 'category':
            # Label encode categorical variables
            X_processed[column] = pd.Categorical(X_processed[column]).codes
        elif pd.api.types.is_bool_dtype(X_processed[column]):
            # Convert boolean values to integers
            X_processed[column] = X_processed[column].astype(int)
    
    # Ensure all columns are numeric
    X_processed = X_processed.astype(float)
    
    # Use Random Forest as the base model for Boruta
    rf = RandomForestClassifier(n_jobs=-1, random_state=42)
    
    # Initialize Boruta
    boruta = BorutaPy(rf, n_estimators='auto', random_state=42)
    
    # Train Boruta
    boruta.fit(X_processed.values, y)
    
    # Get selected and rejected features
    selected_features = boruta.support_
    rejected_features = boruta.support_weak_
    
    # Print feature selection results
    print("\nBoruta Feature Selection Results:")
    print("-" * 40)
    for feature, selected in zip(feature_names, selected_features):
        print(f"{feature:<30} {'Selected' if selected else 'Rejected'}")
    print("-" * 40)
    
    # Get names of selected and rejected features
    selected_feature_names = np.array(feature_names)[selected_features]
    rejected_feature_names = np.array(feature_names)[rejected_features]
    
    print(f"\nSelected {len(selected_feature_names)} features:")
    for feat in selected_feature_names:
        print(f"- {feat}")
    
    print(f"\nRejected {len(rejected_feature_names)} features:")
    for feat in rejected_feature_names:
        print(f"- {feat}")
    
    # Get feature importance rankings
    feature_importance = boruta.ranking_
    
    # Create DataFrame for importance ranking
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance Rank': feature_importance
    }).sort_values('Importance Rank')
    
    # Plot feature importance ranking
    plt.figure(figsize=(12, 6))
    sns.barplot(data=importance_df, x='Importance Rank', y='Feature', palette="Set2")
    plt.title("Feature Importance Rankings", fontsize=16)
    plt.xlabel("Importance Rank (lower is better)", fontsize=12)
    plt.ylabel("Feature", fontsize=12)
    plt.tight_layout()
    plt.show()
    
    return selected_features, selected_feature_names


def balance_samples(features, labels):
    # Get indices for positive and negative samples
    neg_indices = labels[labels == 0].index
    pos_indices = labels[labels == 1].index
    
    # Print original class distribution
    print("\nOriginal Class Distribution:")
    print(labels.value_counts())
    
    # Determine the number of samples in the minority class
    min_samples = min(len(neg_indices), len(pos_indices))
    
    # Undersample the majority class
    if len(neg_indices) > len(pos_indices):
        neg_indices = np.random.choice(neg_indices, min_samples, replace=False)
    else:
        pos_indices = np.random.choice(pos_indices, min_samples, replace=False)
    
    # Combine the balanced indices
    balanced_indices = np.concatenate([neg_indices, pos_indices])
    np.random.shuffle(balanced_indices)
    
    # Extract balanced dataset
    features_balanced = features.loc[balanced_indices]
    labels_balanced = labels.loc[balanced_indices]
    
    # Print class distribution after balancing
    print("\nClass Distribution After Balancing:")
    print(labels_balanced.value_counts())
    
    return features_balanced, labels_balanced


def objective(trial):
    # Sampling hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 200, 500)
    max_depth = trial.suggest_int('max_depth', 4, 12)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_float('max_features', 0.7, 1.0)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    
    # Create the model
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=42
    )
    
    # Repeated Stratified k-fold cross-validation
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    fold_results = []
    gender_labels = features['gender']
    
    # First, balance the entire dataset
    features_balanced, labels_balanced = balance_samples(features, labels)
    
    for fold_idx, (train_index, test_index) in enumerate(rskf.split(features_balanced, labels_balanced, gender_labels)):
        X_train, X_test = features_balanced.iloc[train_index], features_balanced.iloc[test_index]
        y_train, y_test = labels_balanced.iloc[train_index], labels_balanced.iloc[test_index]

        # Output positive/negative sample counts for training and testing sets
        print(f"\nFold {fold_idx + 1}:")
        print("Training set - Positive samples:", sum(y_train == 1), ", Negative samples:", sum(y_train == 0))
        print("Testing set - Positive samples:", sum(y_test == 1), ", Negative samples:", sum(y_test == 0))

        # Perform feature selection using Boruta
        selected_features, selected_feature_names = boruta_feature_selection(X_train, y_train, features_balanced.columns)
        X_train_pls = X_train.loc[:, selected_features]
        X_test_pls = X_test.loc[:, selected_features]

        # Evaluate the model
        results = evaluate_model(model, X_train_pls, y_train, X_test_pls, y_test)
        train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model = results
        
        fold_results.append({
            'fold_idx': fold_idx + 1,
            'train_index': train_index,
            'test_index': test_index,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_sens': train_sens,
            'test_sens': test_sens,
            'train_spec': train_spec,
            'test_spec': test_spec,
            'train_auroc': train_auroc,
            'test_auroc': test_auroc,
            'train_ppv': train_ppv,
            'test_ppv': test_ppv,
            'train_npv': train_npv,
            'test_npv': test_npv,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'selected_feature_names': selected_feature_names,
            'y_test': y_test,
            'y_test_pred': y_test_pred,
            'model': model
        })
    
    # Return average test AUROC as the optimization target for Optuna
    trial.set_user_attr("fold_results", fold_results)
    return np.mean([result['test_auroc'] for result in fold_results])

### PLS feature selection

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from boruta import BorutaPy
import numpy as np
import random


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)



def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    train_sens = recall_score(y_train, y_train_pred)
    test_sens = recall_score(y_test, y_test_pred)
    
    train_spec = recall_score(y_train, y_train_pred, pos_label=0)
    test_spec = recall_score(y_test, y_test_pred, pos_label=0)
    
    train_auroc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    test_auroc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    train_ppv = precision_score(y_train, y_train_pred)
    test_ppv = precision_score(y_test, y_test_pred)
    
    train_npv = precision_score(y_train, y_train_pred, pos_label=0)
    test_npv = precision_score(y_test, y_test_pred, pos_label=0)

    train_f1 = f1_score(y_train, y_train_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_test_proba)
    roc_auc = auc(fpr, tpr)
    
    
    return train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model


def pls_feature_selection(X, y, feature_names):
    pls = PLSRegression(n_components=min(X.shape[1], X.shape[0]-1))
    pls.fit(X, y)
    

    t = pls.x_scores_
    q = pls.y_loadings_
    VIP = np.sqrt(np.sum((t ** 2) @ (q.T ** 2), axis=0) / np.sum(t ** 2, axis=0))
    

    print("\nVIP Scores for each feature:")
    print("-" * 40)
    for feature, vip in zip(feature_names, VIP):
        print(f"{feature:<30} {vip:.3f}")
    print("-" * 40)
    

    print(f"\nVIP Statistics:")
    print(f"Mean VIP: {np.mean(VIP):.3f}")
    print(f"Max VIP: {np.max(VIP):.3f}")
    print(f"Min VIP: {np.min(VIP):.3f}")

    max_vip_index = np.argmax(VIP)  
    max_vip_feature = feature_names[max_vip_index]  

    print(f"Max VIP Feature: {max_vip_feature} with VIP: {np.max(VIP):.3f}")


    print(f"type(VIP): {type(VIP)}")
    print(f"feature_names: {feature_names}")

    if 'gcs' in feature_names:
        gcs_index = feature_names.get_loc('gcs')  
        gcs_vip = VIP[gcs_index]  
        print(f"VIP for 'gcs' feature: {gcs_vip:.3f}")

    else:
        print("'gcs' feature not found in the feature names.")


    selected_features = VIP > 0.5
    if np.sum(selected_features) == 0:
        print("\nWarning: No features with VIP > 1, selecting all features")
        selected_features = np.ones_like(selected_features, dtype=bool)
        
    selected_feature_names = np.array(feature_names)[selected_features]
    print(f"\nSelected {len(selected_feature_names)} features with VIP > 0.5")
    
    return selected_features, selected_feature_names


def balance_samples(features, labels):


    neg_indices = labels[labels == 0].index
    pos_indices = labels[labels == 1].index
    

    print("\nOriginal sample distribution:")
    print(labels.value_counts())
    

    min_samples = min(len(neg_indices), len(pos_indices))
    
 
    if len(neg_indices) > len(pos_indices):
        neg_indices = np.random.choice(neg_indices, min_samples, replace=False)
    else:
        pos_indices = np.random.choice(pos_indices, min_samples, replace=False)

    balanced_indices = np.concatenate([neg_indices, pos_indices])
    np.random.shuffle(balanced_indices)
    

    features_balanced = features.loc[balanced_indices]
    labels_balanced = labels.loc[balanced_indices]
    
 
    print("\nBalanced sample distribution:")
    print(labels_balanced.value_counts())
    
    return features_balanced, labels_balanced



def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 200, 500)
    max_depth = trial.suggest_int('max_depth', 4, 12)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_float('max_features', 0.7, 1.0)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    
 
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=42
    )
    
    # Repeated Stratified k-fold cross-validation
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    fold_results = []
    gender_labels = features['gender']
    

    features_balanced, labels_balanced = balance_samples(features, labels)
    
    for fold_idx, (train_index, test_index) in enumerate(rskf.split(features_balanced, labels_balanced, gender_labels)):
        X_train, X_test = features_balanced.iloc[train_index], features_balanced.iloc[test_index]
        y_train, y_test = labels_balanced.iloc[train_index], labels_balanced.iloc[test_index]


        print(f"\nFold {fold_idx + 1}:")
        print("Training set - Positive samples:", sum(y_train== 1), ", Negative samples:", sum(y_train == 0))
        print("Testing set - Positive samples:", sum(y_test == 1), ", Negative samples:", sum(y_test == 0))


        selected_features, selected_feature_names =pls_feature_selection(X_train, y_train, features_balanced.columns)
        X_train_pls = X_train.loc[:, selected_features]
        X_test_pls = X_test.loc[:, selected_features]


        results = evaluate_model(model, X_train_pls, y_train, X_test_pls, y_test)
        train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model = results
        
        fold_results.append({
            'fold_idx': fold_idx + 1,
            'train_index': train_index,
            'test_index': test_index,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_sens': train_sens,
            'test_sens': test_sens,
            'train_spec': train_spec,
            'test_spec': test_spec,
            'train_auroc': train_auroc,
            'test_auroc': test_auroc,
            'train_ppv': train_ppv,
            'test_ppv': test_ppv,
            'train_npv': train_npv,
            'test_npv': test_npv,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'selected_feature_names': selected_feature_names,
            'y_test': y_test,
            'y_test_pred': y_test_pred,
            'model': model
        })
    
 
    trial.set_user_attr("fold_results", fold_results)
    return np.mean([result['test_auroc'] for result in fold_results])

### GFS feature selection

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from boruta import BorutaPy
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)



def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    train_sens = recall_score(y_train, y_train_pred)
    test_sens = recall_score(y_test, y_test_pred)
    
    train_spec = recall_score(y_train, y_train_pred, pos_label=0)
    test_spec = recall_score(y_test, y_test_pred, pos_label=0)
    
    train_auroc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    test_auroc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    train_ppv = precision_score(y_train, y_train_pred)
    test_ppv = precision_score(y_test, y_test_pred)
    
    train_npv = precision_score(y_train, y_train_pred, pos_label=0)
    test_npv = precision_score(y_test, y_test_pred, pos_label=0)

    train_f1 = f1_score(y_train, y_train_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_test_proba)
    roc_auc = auc(fpr, tpr)
    
    
    return train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model


def gfs_feature_selection(X, y, feature_names, estimator=None, scoring='roc_auc', 
                        max_features=10, min_score_improvement=0.001):
    if estimator is None:
        estimator = DecisionTreeClassifier(
            max_depth=4,
            min_samples_split=20,
            random_state=42
        )

 
    feature_names = list(feature_names)
    selected_features = []
    remaining_features = feature_names.copy()
    best_score = -np.inf

    max_features = min(max_features, len(feature_names))

    while len(selected_features) < max_features and remaining_features:
        scores = []
        for feature in remaining_features:
            candidate_features = selected_features + [feature]
            X_candidate = X[candidate_features]
            
            estimator.fit(X_candidate, y)
            
            if scoring == 'roc_auc':
                probs = estimator.predict_proba(X_candidate)[:, 1]
                score = roc_auc_score(y, probs)
            else:
                score = estimator.score(X_candidate, y)
            
            scores.append((feature, score))
        
        if not scores:
            break
            
        best_feature, best_current_score = max(scores, key=lambda x: x[1])
        
        if (best_current_score - best_score) < min_score_improvement:
            print(f"Stopping: Performance improvement < {min_score_improvement}")
            break
        
        best_score = best_current_score
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        
        print(f"Selected feature: {best_feature} (Score: {best_current_score:.4f})")
    
    if not selected_features:
        print("Warning: No features selected, using first 3 features")
        selected_features = feature_names[:3]
    
    print("\nFinal selected features:")
    for feature in selected_features:
        print(f"- {feature}")
    print(f"Number of selected features: {len(selected_features)}")
    print(f"Best {scoring} score: {best_score:.4f}")
    
    return selected_features, selected_features

def balance_samples(features, labels):


    neg_indices = labels[labels == 0].index
    pos_indices = labels[labels == 1].index
    

    print("\nOriginal sample distribution:")
    print(labels.value_counts())
    

    min_samples = min(len(neg_indices), len(pos_indices))
    

    if len(neg_indices) > len(pos_indices):
        neg_indices = np.random.choice(neg_indices, min_samples, replace=False)
    else:
        pos_indices = np.random.choice(pos_indices, min_samples, replace=False)
    

    balanced_indices = np.concatenate([neg_indices, pos_indices])
    np.random.shuffle(balanced_indices)
    

    features_balanced = features.loc[balanced_indices]
    labels_balanced = labels.loc[balanced_indices]
    

    print("\nBalanced sample distribution:")
    print(labels_balanced.value_counts())
    
    return features_balanced, labels_balanced



def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 200, 500)
    max_depth = trial.suggest_int('max_depth', 4, 12)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_float('max_features', 0.7, 1.0)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=42
    )
    # Repeated Stratified k-fold cross-validation
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    fold_results = []
    gender_labels = features['gender']
    

    features_balanced, labels_balanced = balance_samples(features, labels)
    
    for fold_idx, (train_index, test_index) in enumerate(rskf.split(features_balanced, labels_balanced, gender_labels)):
        X_train, X_test = features_balanced.iloc[train_index], features_balanced.iloc[test_index]
        y_train, y_test = labels_balanced.iloc[train_index], labels_balanced.iloc[test_index]


        print(f"\nFold {fold_idx + 1}:")
        print("Training set - Positive samples:", sum(y_train== 1), ", Negative samples:", sum(y_train == 0))
        print("Testing set - Positive samples:", sum(y_test == 1), ", Negative samples:", sum(y_test == 0))


        selected_features, selected_feature_names =gfs_feature_selection(X_train, y_train, features_balanced.columns)
        X_train_pls = X_train.loc[:, selected_features]
        X_test_pls = X_test.loc[:, selected_features]


        results = evaluate_model(model, X_train_pls, y_train, X_test_pls, y_test)
        train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model = results
        
        fold_results.append({
            'fold_idx': fold_idx + 1,
            'train_index': train_index,
            'test_index': test_index,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_sens': train_sens,
            'test_sens': test_sens,
            'train_spec': train_spec,
            'test_spec': test_spec,
            'train_auroc': train_auroc,
            'test_auroc': test_auroc,
            'train_ppv': train_ppv,
            'test_ppv': test_ppv,
            'train_npv': train_npv,
            'test_npv': test_npv,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'selected_feature_names': selected_feature_names,
            'y_test': y_test,
            'y_test_pred': y_test_pred,
            'model': model
        })
    

    trial.set_user_attr("fold_results", fold_results)
    return np.mean([result['test_auroc'] for result in fold_results])

### Obtain the best features

In [ ]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)

# Retrieve the best trial and its results
best_trial = study.best_trial
best_fold_results = best_trial.user_attrs["fold_results"]

# Find the best fold based on test AUROC
best_fold_idx = np.argmax([result['test_auroc'] for result in best_fold_results])
best_fold_result = best_fold_results[best_fold_idx]

# Print evaluation metrics of the best fold
print("\nBest fold evaluation metrics:")
print(f"  Fold {best_fold_result['fold_idx']}")
print(f"  Train Accuracy: {best_fold_result['train_acc']:.4f}")
print(f"  Test Accuracy: {best_fold_result['test_acc']:.4f}")
print(f"  Train Sensitivity: {best_fold_result['train_sens']:.4f}")
print(f"  Test Sensitivity: {best_fold_result['test_sens']:.4f}")
print(f"  Train Specificity: {best_fold_result['train_spec']:.4f}")
print(f"  Test Specificity: {best_fold_result['test_spec']:.4f}")
print(f"  Train AUROC: {best_fold_result['train_auroc']:.4f}")
print(f"  Test AUROC: {best_fold_result['test_auroc']:.4f}")
print(f"  Train PPV: {best_fold_result['train_ppv']:.4f}")
print(f"  Test PPV: {best_fold_result['test_ppv']:.4f}")
print(f"  Train NPV: {best_fold_result['train_npv']:.4f}")
print(f"  Test NPV: {best_fold_result['test_npv']:.4f}")
print(f"  Train F1 Score: {best_fold_result['train_f1']:.4f}")
print(f"  Test F1 Score: {best_fold_result['test_f1']:.4f}")

# Get the selected features from the best fold
selected_features_final = best_fold_result['selected_feature_names']
print("Selected features for best fold:", selected_features_final)
print(f"Number of selected features: {len(selected_features_final)}")

### The best features

In [ ]:
selected_features_final=['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']

### RF training process

In [ ]:
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.cross_decomposition import PLSRegression
from xgboost import XGBClassifier
import numpy as np
from sklearn.utils import resample

# Function to compute mean and confidence interval
def compute_mean_and_ci(results, metric_idx):
    metrics = np.array([result[metric_idx] for result in results])
    mean = np.mean(metrics)
    ci = 1.96 * np.std(metrics) / np.sqrt(len(metrics))  # 95% Confidence Interval
    return mean, ci

# Define list of evaluation metrics
metrics = ['Train Accuracy', 'Test Accuracy', 'Train Sensitivity', 'Test Sensitivity',
           'Train Specificity', 'Test Specificity', 'Train AUROC', 'Test AUROC',
           'Train PPV', 'Test PPV', 'Train NPV', 'Test NPV', 'Train F1 Score', 'Test F1 Score']

# Retrain the model with best parameters and selected features using 5-fold cross-validation
best_params = best_trial.params
model = RandomForestClassifier(**best_params)
print("best_params:", best_params)

# Get selected features
# selected_features_final = best_fold_result['selected_feature_names']
print("Selected features:", selected_features_final)
print(f"Number of selected features: {len(selected_features_final)}")

# Select features using pandas column names instead of positional indexing
X_final = features[selected_features_final]
gender_labels = features['gender']

# Repeated Stratified K-Fold
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
final_fold_results = []
y_test_all = []
y_test_pred_all = []
final_y_pred_prob = []

def balance_samples(X, y, random_state=42):
    """Balance positive and negative samples"""
    # Convert y to DataFrame if it's a Series
    if isinstance(y, pd.Series):
        y = pd.DataFrame(y)
    else:
        y = pd.DataFrame(y, columns=['target'])

    # Combine X and y
    X_y = pd.concat([X.reset_index(drop=True), y.reset_index(drop=True)], axis=1)

    # Get label column name
    y_col = y.columns[0]

    # Separate positive and negative samples
    negative_samples = X_y[X_y[y_col] == 0]
    positive_samples = X_y[X_y[y_col] == 1]

    # Get the number of samples in the minority class
    min_samples = min(len(negative_samples), len(positive_samples))

    # Undersample the majority class
    if len(negative_samples) > min_samples:
        negative_samples = resample(negative_samples,
                                    n_samples=min_samples,
                                    random_state=random_state)
    if len(positive_samples) > min_samples:
        positive_samples = resample(positive_samples,
                                    n_samples=min_samples,
                                    random_state=random_state)

    # Combine balanced samples
    balanced_samples = pd.concat([negative_samples, positive_samples])

    # Shuffle the data
    balanced_samples = balanced_samples.sample(frac=1, random_state=random_state)

    # Split back into features and labels
    X_balanced = balanced_samples[X.columns]
    y_balanced = balanced_samples[y_col]

    return X_balanced, y_balanced


# Perform repeated stratified k-fold cross-validation ensuring gender and label stratification
for fold_idx, (train_index, test_index) in enumerate(rskf.split(X_final, labels, gender_labels)):
    X_train_fold_orig, X_test_fold = X_final.iloc[train_index], X_final.iloc[test_index]
    y_train_fold_orig, y_test_fold = labels.iloc[train_index], labels.iloc[test_index]

    # Balance training set
    X_train_fold, y_train_fold = balance_samples(X_train_fold_orig, y_train_fold_orig)
    # Balance test set
    X_test_fold, y_test_fold = balance_samples(X_test_fold, y_test_fold)

    # Output sample counts for current fold
    print(f"\nFold {fold_idx + 1}:")
    print("Training set - Positive samples:", sum(y_train_fold == 1), ", Negative samples:", sum(y_train_fold == 0))
    print("Testing set - Positive samples:", sum(y_test_fold == 1), ", Negative samples:", sum(y_test_fold == 0))

    # Print features used in this fold
    print("\nFeatures used in this fold:")
    for i, feature in enumerate(X_train_fold.columns, 1):
        print(f"{i}. {feature}")
    print("-" * 50)  # Divider for better readability

    # Train the model and get prediction probabilities
    model.fit(X_train_fold, y_train_fold)
    y_test_prob = model.predict_proba(X_test_fold)[:, 1]

    # Collect true labels and predicted probabilities
    y_test_all.extend(y_test_fold)
    final_y_pred_prob.extend(y_test_prob)

    # Evaluate the model
    final_results = evaluate_model(model, X_train_fold, y_train_fold, X_test_fold, y_test_fold)
    final_fold_results.append(final_results)

# Generate final predictions based on probability threshold
final_y_pred = np.array([1 if prob >= 0.5 else 0 for prob in final_y_pred_prob])

# Output length check
print(f"\nTotal true labels: {len(y_test_all)}")
print(f"Total predicted probabilities: {len(final_y_pred_prob)}")

# Print evaluation metrics for each fold
for idx, result in enumerate(final_fold_results, 1):
    train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model = result
    print(f"\nFold {idx} evaluation metrics after refitting:")
    print(f"  Train Accuracy: {train_acc:.4f}")
    print(f"  Test Accuracy: {test_acc:.4f}")
    print(f"  Train Sensitivity: {train_sens:.4f}")
    print(f"  Test Sensitivity: {test_sens:.4f}")
    print(f"  Train Specificity: {train_spec:.4f}")
    print(f"  Test Specificity: {test_spec:.4f}")
    print(f"  Train AUROC: {train_auroc:.4f}")
    print(f"  Test AUROC: {test_auroc:.4f}")
    print(f"  Train PPV: {train_ppv:.4f}")
    print(f"  Test PPV: {test_ppv:.4f}")
    print(f"  Train NPV: {train_npv:.4f}")
    print(f"  Test NPV: {test_npv:.4f}")
    print(f"  Train F1: {train_f1:.4f}")
    print(f"  Test F1: {test_f1:.4f}")

# Compute and print average performance across folds
print("\nOverall Performance:")
for idx, metric in enumerate(metrics):
    mean, ci = compute_mean_and_ci(final_fold_results, idx)
    print(f'{metric}: Mean = {mean:.3f}, 95% CI = [{mean - ci:.3f}, {mean + ci:.3f}]')

### RF-SHAP summary

In [ ]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# Global plot settings
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.labelsize'] = 26
plt.rcParams['xtick.labelsize'] = 26
plt.rcParams['ytick.labelsize'] = 26
plt.rcParams['legend.fontsize'] = 26
FEATURE_FONT_SIZE = 14

def create_shap_plots(X_resampled, y_resampled, selected_features_final, best_params, rskf):
    """Generate SHAP plots for each fold"""
    
    for fold_idx, (train_index, test_index) in enumerate(rskf.split(X_resampled, y_resampled)):
        print(f'Fold {fold_idx+1}: SHAP values')
        
        # Get fold data
        X_train_fold_orig = X_resampled.iloc[train_index]
        y_train_fold_orig = y_resampled[train_index]
        X_test_fold_orig = X_resampled.iloc[test_index]
        y_test_fold_orig = y_resampled[test_index]
        
        # Balance datasets
        X_train_fold, y_train_fold = balance_samples(X_train_fold_orig, y_train_fold_orig)
        X_test_fold, y_test_fold = balance_samples(X_test_fold_orig, y_test_fold_orig)
        
        try:
            # Train model with best parameters
            model = RandomForestClassifier(**best_params)
            model.fit(X_train_fold, y_train_fold)
            
            # Calculate SHAP values
            explainer = shap.TreeExplainer(model, feature_perturbation='interventional')
            shap_values = explainer.shap_values(X_test_fold)
            
            # Ensure correct shape for plotting
            if isinstance(shap_values, np.ndarray) and len(shap_values.shape) == 3:
                shap_values = [shap_values[:,:,0], shap_values[:,:,1]]
            
            # Debug SHAP values
            print("Number of features:", len(X_test_fold.columns))
            print("SHAP values type:", type(shap_values))
            if isinstance(shap_values, list):
                print("SHAP values shapes:", [sv.shape for sv in shap_values])
            else:
                print("SHAP values shape:", shap_values.shape)

            # Print shapes for debugging
            print("SHAP_value shape:", np.array(shap_values).shape if isinstance(shap_values, list) else shap_values.shape)
            print("X_test shape:", X_test_fold.shape)
            print("Features:", list(X_test_fold.columns))
            
            # Summary plot
            plt.figure(figsize=(12, 8))
            with plt.style.context({'font.family': 'Times New Roman'}):
                shap_values_plot = shap_values[1] if isinstance(shap_values, list) else shap_values
                shap.summary_plot(
                    shap_values_plot,
                    X_test_fold,
                    show=False,
                    plot_size=(10, 6),
                    max_display=len(X_test_fold.columns)  
                )
                
                ax = plt.gca()
                ax.set_title(f'SHAP Summary Plot (Top 10) - Fold {fold_idx+1}', 
                           fontfamily='Times New Roman', 
                           fontsize=12, 
                           pad=20)
                ax.set_xlabel(ax.get_xlabel(), fontfamily='Times New Roman', fontsize=14)
                ax.set_ylabel(ax.get_ylabel(), fontfamily='Times New Roman', fontsize=12)
                ax.tick_params(axis='y', labelsize=FEATURE_FONT_SIZE)
                
                plt.tight_layout()
                plt.show()
            
            # Bar plot
            plt.figure(figsize=(10, 6))
            with plt.style.context({'font.family': 'Times New Roman'}):
                shap.summary_plot(
                    shap_values_plot,
                    X_test_fold,
                    plot_type="bar",
                    show=False,
                    plot_size=(10, 6),
                    max_display=len(X_test_fold.columns)  
                )
             
                ax = plt.gca()
                bars = ax.patches  
                for bar in bars:
                    bar.set_facecolor('blue')  
                ax.set_title(f'SHAP Bar Plot (Top 10) - Fold {fold_idx+1}', 
                fontfamily='Times New Roman', fontsize=12, pad=20)
                ax.set_xlabel(ax.get_xlabel(), fontfamily='Times New Roman', fontsize=14)
                ax.set_ylabel(ax.get_ylabel(), fontfamily='Times New Roman', fontsize=12)
                ax.tick_params(axis='both', which='major', labelsize=16, labelrotation=0)
                ax.tick_params(axis='y', labelsize=FEATURE_FONT_SIZE)
                
                plt.tight_layout()
                plt.show()
            
            # Force plot for example instances
            positive_indices = np.where(y_resampled == 1)[0]
            negative_indices = np.where(y_resampled == 0)[0]
            
            if len(positive_indices) > 0 and len(negative_indices) > 0:
                first_positive = X_test_fold.iloc[0]
                first_negative = X_test_fold.iloc[-1]
                
                shap_values_pos = explainer.shap_values(first_positive)
                shap_values_neg = explainer.shap_values(first_negative)
                
                plt.figure(figsize=(12, 4))
                shap.force_plot(
                    explainer.expected_value,
                    shap_values_pos[1] if isinstance(shap_values_pos, list) else shap_values_pos,
                    first_positive,
                    show=False,
                    matplotlib=True
                )
                plt.title("Force Plot - Positive Example", fontfamily='Times New Roman', fontsize=12)
                plt.tight_layout()
                plt.show()
            
        except Exception as e:
            print(f"Error in fold {fold_idx+1}:", str(e))
            print("Shapes:", X_train_fold.shape, y_train_fold.shape, X_test_fold.shape, y_test_fold.shape)
            break

# Usage
X_resampled = X_final[selected_features_final]
create_shap_plots(X_resampled, labels, selected_features_final, best_params, rskf)



### RF-CIC curve chart

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def calculate_metrics_at_threshold(y_true, y_prob, threshold):
    """Calculate performance metrics at a given threshold"""
    y_pred = (y_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    total_samples = len(y_true)
    scaling_factor = 1000 / total_samples
    
    n_high_risk = (fp + tp) * scaling_factor
    n_high_risk_with_event = tp * scaling_factor
    
    return n_high_risk, n_high_risk_with_event

def plot_cost_benefit_curve(y_true, y_prob):
    """Plot cost-benefit curve"""
    # Create a smaller figure with more space at the bottom
    fig, ax = plt.subplots(figsize=(4, 3.2), dpi=300)
    
    # Set background color to white
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    
    # Generate threshold sequence
    thresholds = np.linspace(0, 1, 1000)
    
    # Calculate metrics
    n_high_risk_list = []
    n_high_risk_with_event_list = []
    
    for threshold in thresholds:
        n_high_risk, n_high_risk_with_event = calculate_metrics_at_threshold(
            y_true, y_prob, threshold
        )
        n_high_risk_list.append(n_high_risk)
        n_high_risk_with_event_list.append(n_high_risk_with_event)
    
    # Plot main curves with labels
    ax.plot(thresholds, n_high_risk_list, color='red', linestyle='-', 
            linewidth=1.0, label='Number high risk')
    ax.plot(thresholds, n_high_risk_with_event_list, color='blue', 
            linestyle='--', linewidth=1.0, label='Number high risk with event')
    
    # Add legend
    ax.legend(fontsize=8, loc='upper right', frameon=False)
    
    # Set axis labels with smaller font
    ax.set_xlabel('High Risk Threshold', fontsize=9)
    ax.set_ylabel('Number high risk (out of 1000)', fontsize=9)
    
    # Set axis limits
    ax.set_ylim(0, 1000)
    ax.set_xlim(0, 1)
    
    # Set smaller tick font size
    ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Add cost-benefit ratio labels
    cost_benefit_ratios = ['1:100', '1:4', '2:3', '3:2', '4:1', '100:1']
    ratio_positions = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    
    # Create secondary x-axis at the bottom
    ax2 = ax.secondary_xaxis(-0.25)
    ax2.set_xlim(ax.get_xlim())
    ax2.set_xticks(ratio_positions)
    ax2.set_xticklabels(cost_benefit_ratios, fontsize=8)
    
    # Set label for secondary axis
    ax2.set_xlabel('Cost:Benefit Ratio', fontsize=9, labelpad=8)
    
    # Set tick parameters for secondary axis
    ax2.tick_params(axis='both', which='major', labelsize=8)
    
    # Set border line width
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)
    
    # Hide unnecessary spines for ax2
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    
    # Adjust margins to make room for bottom axis
    plt.subplots_adjust(bottom=0.25)
    
    return plt

# Use real prediction results from the model
y_true = np.array(y_test_all)
y_prob = np.array(final_y_pred_prob)

# Plot the curve
plt = plot_cost_benefit_curve(y_true, y_prob)

# Save the smaller figure
plt.savefig('cost_benefit_curve.png', dpi=300, bbox_inches='tight', 
            format='png')
plt.show()

### six machine learning models training

In [ ]:
# Import required libraries
import optuna
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import (accuracy_score, roc_auc_score, confusion_matrix, 
                           precision_score, recall_score, f1_score)
import numpy as np

# XGBoost model selected features and hyperparameters
# best_params: {'n_estimators': 201, 'max_depth': 9, 'learning_rate': 0.1335376701461377, 'min_child_weight': 6, 
# 'gamma': 0.21064265390797143, 'subsample': 0.895162492770872, 'colsample_bytree': 0.7272554168734067, 
# 'scale_pos_weight': 1.1277357372234167, 'reg_alpha': 0.5503700132271422, 'reg_lambda': 0.6075194736990024}
# Selected features: ['age' 'aniongap' 'bun' 'creatinine' 'wbc' 'rbc' 'weight'
#  'hr' 'rr' 'temperature' 'glucose' 'gcs']
# Number of selected features: 12
def objective_xgboost(trial):
    # Use the best parameters provided in comments
    params = {
        'n_estimators': 201,
        'max_depth': 9,
        'learning_rate': 0.1335376701461377,
        'min_child_weight': 6,
        'gamma': 0.21064265390797143,
        'subsample': 0.895162492770872,
        'colsample_bytree': 0.7272554168734067,
        'scale_pos_weight': 1.1277357372234167,
        'reg_alpha': 0.5503700132271422,
        'reg_lambda': 0.6075194736990024,
        'random_state': 42
    }
    Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    return params, Selected_features


# LightGBM model selected features and hyperparameters
# best_params: {'learning_rate': 0.047669762018487424, 'n_estimators': 79, 'num_leaves': 45, 'max_depth': 7, 
# 'min_child_samples': 7, 'feature_fraction': 0.8057018384484658, 'bagging_fraction': 0.706785747058316}
# Selected features: ['age' 'aniongap' 'bun' 'creatinine' 'wbc' 'rbc' 'hr'
#  'rr' 'temperature' 'glucose' 'gcs']
# Number of selected features: 11
def objective_lightgbm(trial):
    # Use the best parameters provided in comments
    params = {
        'learning_rate': 0.047669762018487424,
        'n_estimators': 79,
        'num_leaves': 45,
        'max_depth': 7,
        'min_child_samples': 7,
        'feature_fraction': 0.8057018384484658,
        'bagging_fraction': 0.706785747058316,
        'random_state': 42
    }
    Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    return params, Selected_features


# Random Forest model selected features and hyperparameters
# best_params: {'n_estimators': 141, 'max_depth': 9, 
#               'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'log2'}
# Selected features: ['age' 'aniongap' 'bun' 'creatinine' 'wbc' 'rbc' 'weight'
#  'hr' 'rr' 'temperature' 'glucose' 'gcs']
# Number of selected features: 12
def objective_random_forest(trial):
    # Use the best parameters provided in comments
    params = {
        'n_estimators': 141,
        'max_depth': 9,
        'min_samples_split': 10,
        'min_samples_leaf': 6,
        'max_features': 'log2',
        'random_state': 42
    }
    Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    return params, Selected_features


# SVM model selected features and hyperparameters
# best_params: {'C': 3.8606754376312318}
# Selected features: ['age' 'aniongap' 'bun' 'creatinine' 'wbc' 'rbc' 'weight'
#  'hr' 'rr' 'temperature' 'glucose' 'gcs']
# Number of selected features: 12
def objective_svm(trial):
    params = {
        'C': 3.8606754376312318,
        'gamma': 'scale',
        'kernel': 'rbf',
        'probability': True,
        'random_state': 42,
        'cache_size': 1000
    }
    Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    return params, Selected_features


# AdaBoost model selected features and hyperparameters
# best_params: {'n_estimators': 177, 'learning_rate': 0.420580705420142}
# Selected features: ['aniongap' 'bun' 'creatinine' 'wbc' 'rbc' 'hr' 'rr'
#  'temperature' 'glucose' 'gcs']
# Number of selected features: 10
def objective_adaboost(trial):
    params = {
        'n_estimators': 177,
        'learning_rate': 0.420580705420142,
        'random_state': 42
    }
    Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    return params, Selected_features


# Logistic Regression model selected features and hyperparameters
# best_params: {'C': 2.195022414600841, 'penalty': 'l1', 'solver': 'liblinear'}
# Selected features: ['age' 'aniongap' 'bun' 'creatinine' 'wbc' 'rbc' 'hr'
#  'rr' 'temperature' 'glucose' 'gcs']
# Number of selected features: 11
def objective_logistic(trial):
    params = {
        'C': 1.9351172214826404,
        'penalty': 'l2',
        'solver': 'liblinear',
        'max_iter': 500,
        'random_state': 42
    }
    Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    return params, Selected_features


# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred, y_prob):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    ppv = precision_score(y_true, y_pred)
    npv = tn / (tn + fn)
    f1 = f1_score(y_true, y_pred)
    auroc = roc_auc_score(y_true, y_prob)
    return {
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'AUROC': auroc,
        'PPV': ppv,
        'NPV': npv,
        'F1 Score': f1
    }


# Get optimized parameters and features for each model
xgb_params, xgb_features = objective_xgboost(None)
lgb_params, lgb_features = objective_lightgbm(None)
rf_params, rf_features = objective_random_forest(None)
svm_params, svm_features = objective_svm(None)
ada_params, ada_features = objective_adaboost(None)
lr_params, lr_features = objective_logistic(None)

# Define models with optimized parameters
models = {
    'XGBoost': {'model': XGBClassifier(**xgb_params), 'features': xgb_features},
    'LightGBM': {'model': LGBMClassifier(**lgb_params), 'features': lgb_features},
    'AdaBoost': {'model': AdaBoostClassifier(**ada_params), 'features': ada_features},
    'RandomForest': {'model': RandomForestClassifier(**rf_params), 'features': rf_features},
    'SVM': {'model': SVC(**svm_params), 'features': svm_features},
    'LogisticRegression': {'model': LogisticRegression(**lr_params), 'features': lr_features},
}

model_name = ['XGBoost','LightGBM','RandomForest','AdaBoost','SVM','AdaBoost','LogisticRegression']

# Dictionary to store evaluation metrics for all models
all_models_metrics = {model_name: {
    'train_metrics': [],
    'test_metrics': []
} for model_name in models.keys()}


# Function to compute mean and confidence interval
def compute_mean_and_ci(results, metric_idx):
    metrics = np.array([result[metric_idx] for result in results])
    mean = np.mean(metrics)
    ci = 1.96 * np.std(metrics) / np.sqrt(len(metrics))  # 95% confidence interval
    return mean, ci


# List of evaluation metrics
metrics = ['Train Accuracy', 'Test Accuracy', 'Train Sensitivity', 'Test Sensitivity', 
          'Train Specificity', 'Test Specificity', 'Train AUROC', 'Test AUROC', 
          'Train PPV', 'Test PPV', 'Train NPV', 'Test NPV', 'Train F1 Score', 'Test F1 Score']


# Dictionary to store evaluation metrics for all models
all_models_metrics = {model_name: {
    'train_metrics': [],
    'test_metrics': []
} for model_name in models.keys()}


# Train and evaluate each model
for model_name, model_info in models.items():
    print(f"\nTraining {model_name}...")
    model = model_info['model']
    features = model_info['features']
    
    # Check if features exist
    available_features = []
    missing_features = []
    for feature in features:
        if feature in X_final.columns:
            available_features.append(feature)
        else:
            missing_features.append(feature)
    if missing_features:
        print(f"Warning: Following features are missing for {model_name}: {missing_features}")
        print(f"Will proceed with available features: {available_features}")
        features = available_features
    
    # Reset result storage for each model
    final_fold_results = []
    y_test_all = []
    y_test_pred_all = []
    final_y_pred_prob = []

    # Create subset with selected features
    X_selected = X_final[features]
    print("Available features in dataset:")
    print(X_final.columns.tolist())

    # Repeated Stratified K-Fold
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    for fold_idx, (train_index, test_index) in enumerate(rskf.split(X_selected, labels, gender_labels)):
        print(f"\nFold {fold_idx + 1}:")
        X_train_fold_orig = X_selected.iloc[train_index]
        X_test_fold = X_selected.iloc[test_index]
        y_train_fold_orig = labels.iloc[train_index]
        y_test_fold = labels.iloc[test_index]

        # Balance samples
        X_train_fold, y_train_fold = balance_samples(X_train_fold_orig, y_train_fold_orig)
        X_test_fold, y_test_fold = balance_samples(X_test_fold, y_test_fold)

        # Output number of positive/negative samples in current fold
        print("Training set - Positive samples:", sum(y_train_fold == 1), ", Negative samples:", sum(y_train_fold == 0))
        print("Testing set - Positive samples:", sum(y_test_fold == 1), ", Negative samples:", sum(y_test_fold == 0))

        # Print features used in this fold
        print("\nFeatures used in this fold:")
        for i, feature in enumerate(X_train_fold.columns, 1):
            print(f"{i}. {feature}")
        print("-" * 50)

        # Train and evaluate the model
        final_results = evaluate_model(model, X_train_fold, y_train_fold, X_test_fold, y_test_fold)
        final_fold_results.append(final_results)

        # Collect prediction results
        y_test_all.extend(y_test_fold)
        if isinstance(model, SVC):
            y_test_prob = model.decision_function(X_test_fold)
            y_test_prob = 1 / (1 + np.exp(-y_test_prob))
        else:
            y_test_prob = model.predict_proba(X_test_fold)[:, 1]
        final_y_pred_prob.extend(y_test_prob)

    # Final output of label lengths
    print(f"\nTotal true labels: {len(y_test_all)}")
    print(f"Total predicted probabilities: {len(final_y_pred_prob)}")

    # Print evaluation metrics for each fold
    for idx, result in enumerate(final_fold_results, 1):
        train_acc, test_acc, train_sens, test_sens, train_spec, test_spec, train_auroc, test_auroc, train_ppv, test_ppv, train_npv, test_npv, train_f1, test_f1, y_test, y_test_pred, model = result
        print(f"\nFold {idx} evaluation metrics after refitting:")
        print(f"  Train Accuracy: {train_acc:.4f}")
        print(f"  Test Accuracy: {test_acc:.4f}")
        print(f"  Train Sensitivity: {train_sens:.4f}")
        print(f"  Test Sensitivity: {test_sens:.4f}")
        print(f"  Train Specificity: {train_spec:.4f}")
        print(f"  Test Specificity: {test_spec:.4f}")
        print(f"  Train AUROC: {train_auroc:.4f}")
        print(f"  Test AUROC: {test_auroc:.4f}")
        print(f"  Train PPV: {train_ppv:.4f}")
        print(f"  Test PPV: {test_ppv:.4f}")
        print(f"  Train NPV: {train_npv:.4f}")
        print(f"  Test NPV: {test_npv:.4f}")
        print(f"  Train F1: {train_f1:.4f}")
        print(f"  Test F1: {test_f1:.4f}")

    # Store results for this model
    all_models_metrics[model_name] = {
        'final_fold_results': final_fold_results,
        'y_test_all': y_test_all,
        'final_y_pred_prob': final_y_pred_prob
    }

# Print comparison results for all models
print("\nModel Comparison:")
for model_name in models.keys():
    results = all_models_metrics[model_name]['final_fold_results']
    print(f"\n{model_name}:")
    for idx, metric in enumerate(metrics):
        mean, ci = compute_mean_and_ci(results, idx)
        print(f'{metric}: Mean = {mean:.3f}, 95% CI = [{mean-ci:.3f}, {mean+ci:.3f}]')



### ROC curve of six machine learning models

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

def plot_roc_curves(models, all_models_metrics):

   plt.rcParams.update({
       'font.family': 'Times New Roman',
       'font.serif': ['Times New Roman'],
       'font.size': 12,
       'axes.labelsize': 16, 
       'axes.titlesize': 16,
       'xtick.labelsize': 16,
       'ytick.labelsize': 16,
       'legend.fontsize': 14
   })

   plt.figure(figsize=(8, 6), dpi=300)

   colors = {
       'XGBoost': '#4B7BE5',
       'LightGBM': '#2E8B57',
       'RandomForest': '#DC3445',
        'AdaBoost': '#A0522D',
        'SVM': '#FFA500',
       'LogisticRegression': '#9B51E0', 
   }


   model_aucs = {}
   for model_name in models.keys():
       results = all_models_metrics[model_name]['final_fold_results']
       test_aurocs = [result[7] for result in results]
       mean_auc = np.mean(test_aurocs)
       std_auc = np.std(test_aurocs)
       model_aucs[model_name] = (mean_auc, std_auc)

   ax = plt.gca()
   ax.set_facecolor('white')
   plt.gcf().set_facecolor('white')

   for model_name in models.keys():
       y_test = all_models_metrics[model_name]['y_test_all']
       y_pred_prob = all_models_metrics[model_name]['final_y_pred_prob']
       
       fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
       mean_auc, std_auc = model_aucs[model_name]
       
       plt.plot(fpr, tpr,
                color=colors[model_name],
                label=f'{model_name} (AUC = {mean_auc:.3f} ± {std_auc:.3f})',
                linewidth=1.5)

   plt.plot([0, 1], [0, 1], color='gray', linestyle='--', linewidth=1.5)
   plt.xlim([0, 1])
   plt.ylim([0, 1])

   plt.xlabel('False Positive Rate (1 - Specificity)', fontfamily='Times New Roman')
   plt.ylabel('True Positive Rate (Sensitivity)', fontfamily='Times New Roman')
   plt.title('Average ROC Curves for Different Models', fontfamily='Times New Roman', pad=10)

   plt.legend(loc="lower right",
             prop={'family': 'Times New Roman'},
             frameon=False,
             edgecolor='black')

   plt.xticks(fontfamily='Times New Roman')
   plt.yticks(fontfamily='Times New Roman')

   for spine in ax.spines.values():
       spine.set_linewidth(1)

   #plt.grid(True, linestyle=':', alpha=0.3)
   plt.tight_layout()
   return plt

plt = plot_roc_curves(models, all_models_metrics)  
plt.savefig('roc_curves_mimic_predict_eicu.png', dpi=600, bbox_inches='tight')
plt.show()

### Confusion matrix diagram of six machine learning models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(all_models_metrics):
    plt.rcParams.update({
        'font.family': 'Times New Roman',
        'font.serif': ['Times New Roman'],
        'font.size': 12,
        'axes.labelsize': 16,
        'axes.titlesize': 16,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 14
    })

    fig, axes = plt.subplots(2, 3, figsize=(15, 10), dpi=300)
    fig.suptitle('Confusion Matrices for Different Models', 
                 fontsize=20, 
                 y=1.05, 
                 fontfamily='Times New Roman')

    cmap = plt.cm.Blues

    for idx, (model_name, metrics) in enumerate(all_models_metrics.items()):
        row = idx // 3
        col = idx % 3
        ax = axes[row, col]
        ax.set_facecolor('white')
        
        # 从metrics中获取真实标签和预测概率
        y_true = np.array(metrics['y_test_all'])
        y_pred_prob = np.array(metrics['final_y_pred_prob'])
        fold_size = len(y_true) // 5
        
        cm_sum = np.zeros((2, 2))
        
        for i in range(5):
            start_idx = i * fold_size
            end_idx = (i + 1) * fold_size if i < 4 else len(y_true)
            fold_y_true = y_true[start_idx:end_idx]
            fold_y_pred = (y_pred_prob[start_idx:end_idx] > 0.5).astype(int)
            cm = confusion_matrix(fold_y_true, fold_y_pred)
            cm_sum += cm
        
        cm = (cm_sum / 5).astype(int)
        total = cm.sum()
        cm_percent = cm / total * 100
        cm_norm = cm.astype('float') / cm.max()
        
        im = ax.imshow(cm_norm, interpolation='nearest', 
                      cmap=cmap,
                      aspect='auto')
        
        # 修改这里：对于浅色背景使用黑色文字，深色背景使用白色文字
        for i in range(2):
            for j in range(2):
                # 调整阈值，使得浅色区域都使用黑色文字
                color = "white" if cm_norm[i, j] > 0.7 else "black"
                # 显示数值
                ax.text(j, i, str(cm[i, j]),
                       ha="center", va="center",
                       color=color,
                       fontsize=14,
                       fontfamily='Times New Roman',
                       fontweight='bold')
                # 显示百分比
                ax.text(j, i + 0.25,
                       f'({cm_percent[i,j]:.1f}%)',
                       ha='center', 
                       va='center',
                       color=color,
                       fontsize=12,
                       fontfamily='Times New Roman')
        
        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])
        ax.set_xticklabels(['Predicted\nNegative', 'Predicted\nPositive'],
                          fontfamily='Times New Roman', 
                          fontsize=14)
        ax.set_yticklabels(['Actual\nNegative', 'Actual\nPositive'],
                          fontfamily='Times New Roman', 
                          fontsize=14)
        
        ax.set_title(f'{model_name}', 
                    pad=10, 
                    fontsize=16, 
                    fontfamily='Times New Roman')
        
        for spine in ax.spines.values():
            spine.set_linewidth(1.0)
            
        ax.set_xticks(np.arange(-0.5, 2, 1), minor=True)
        ax.set_yticks(np.arange(-0.5, 2, 1), minor=True)
        ax.grid(which="minor", color="black", linestyle='-', linewidth=1)
        ax.tick_params(which="minor", size=0)

    fig.patch.set_facecolor('white')
    plt.tight_layout()
    plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
    
    return plt

# example:
plt = plot_confusion_matrices(all_models_metrics)
plt.show()

### DCA decision curve graph of six machine learning models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_dca_curves(all_models_metrics, title="Model Comparison - DCA Curves"):
    plt.figure(figsize=(8, 6))

    # Set global font settings
    plt.rcParams.update({
        'font.family': 'Times New Roman',
        'font.serif': ['Times New Roman'],
        'font.size': 12,
        'axes.labelsize': 16, 
        'axes.titlesize': 16,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 14
    })

    # Define color mapping for models
    colors = {
        'XGBoost': '#2E86C1',
        'LightGBM': '#27AE60',
        'RandomForest': '#8E44AD',
        'SVM': '#E67E22',
        'AdaBoost': '#C0392B',
        'LogisticRegression': '#34495E'
    }
    
    # Generate threshold probabilities
    thresholds = np.arange(0, 1.01, 0.01)
    
    # Calculate and plot DCA curves for each model
    for model_name, metrics in all_models_metrics.items():
        y_true = np.array(metrics['y_test_all'])
        y_pred_prob = np.array(metrics['final_y_pred_prob'])
        
        # Ensure the lengths are consistent
        min_len = min(len(y_true), len(y_pred_prob))
        y_true = y_true[:min_len]
        y_pred_prob = y_pred_prob[:min_len]
        
        # Compute net benefit for each threshold
        net_benefits = []
        for threshold in thresholds:
            y_pred = (y_pred_prob >= threshold).astype(int)
            TP = np.sum((y_true == 1) & (y_pred == 1))
            FP = np.sum((y_true == 0) & (y_pred == 1))
            n = len(y_true)
            
            if TP + FP == 0:
                net_benefit = 0
            else:
                net_benefit = (TP / n) - (FP / n) * (threshold / (1 - threshold))
            net_benefits.append(net_benefit)
            
        plt.plot(thresholds, net_benefits, '-', color=colors[model_name], 
                 linewidth=1.5, label=model_name)
    
    # Plot baseline: treat-all and treat-none strategies
    y_true = np.array(metrics['y_test_all'])  # Reuse from last model
    all_positives = np.ones_like(y_true)
    net_benefits_all = []
    for threshold in thresholds:
        TP = np.sum(y_true == 1)
        FP = np.sum(y_true == 0)
        n = len(y_true)
        net_benefit = (TP / n) - (FP / n) * (threshold / (1 - threshold))
        net_benefits_all.append(net_benefit)
    
    plt.plot(thresholds, net_benefits_all, '--', color='black', 
             linewidth=1.5, label='Treat All')
    plt.plot(thresholds, np.zeros_like(thresholds), '--', color='gray', 
             linewidth=1.5, label='Treat None')
    
    # Set axis labels
    plt.xlabel('Threshold Probability', fontsize=16)
    plt.ylabel('Net Benefit', fontsize=16)
    plt.title(title, fontsize=16, pad=15)

    # Set tick label sizes
    plt.tick_params(axis='x', labelsize=16)
    plt.tick_params(axis='y', labelsize=16)

    # Set legend
    plt.legend(loc='upper right', frameon=False, facecolor='white', 
               edgecolor='black', fontsize=14)
    
    # Set axis limits
    plt.xlim(0, 1)
    plt.ylim(min(min(net_benefits), -0.05), 
             max(max(net_benefits_all), max(net_benefits)) + 0.05)
    
    # Adjust layout
    plt.tight_layout()

    # Set border line width
    for spine in plt.gca().spines.values():
        spine.set_linewidth(1.5)

    return plt.gcf()

# Example usage:
plot_dca_curves(all_models_metrics, "Model Comparison - DCA Curves")
plt.savefig('dca_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

### Load eICU data

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import (accuracy_score, roc_auc_score, confusion_matrix, 
                             precision_score, recall_score, f1_score)
import numpy as np
import pandas as pd
from sklearn.utils import resample
import joblib
from xgboost import XGBClassifier

# Read the dataset
file_path = "E:\\eicu_data.csv"
data = pd.read_csv(file_path)

# Print number of samples before preprocessing
print("Number of samples before data preprocessing:", len(data))

# Data preprocessing
filtered_data1 = data[data['age'] >= 18]
print("Number of samples after data preprocessing (age >= 18):", len(filtered_data1))

filtered_data2 = filtered_data1[filtered_data1['myocardial_infarct'] != 1]

print("Number of samples after data preprocessing (non-myocardial infarction):", len(filtered_data2))

filtered_data3 = filtered_data2[filtered_data2['congestive_heart_failure'] != 1]
print("Number of samples after data preprocessing (non-congestive heart failure):", len(filtered_data3))

filtered_data4 = filtered_data3[filtered_data3['icu_los_hours'] >= 3]
print("Number of samples after data preprocessing (ICU length of stay >= 3 hours):", len(filtered_data4))

# Final processed dataset
external_filtered_data = filtered_data4
print("Final number of samples after data preprocessing:", len(external_filtered_data))

# Print label distribution in the final dataset
label_distribution = external_filtered_data['label'].value_counts()
print("Label distribution in the final dataset:")
print(label_distribution)



### Features obtained from MIMIC

In [ ]:
Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
# Extract features and labels
X = external_filtered_data[Selected_features]  
y = external_filtered_data['label']


### Test set-evaluation results

In [ ]:
from sklearn.impute import SimpleImputer
from scipy import stats

# XGBoost trained hyperparameters
best_params = {'n_estimators': 255, 'max_depth': 10, 'learning_rate': 0.09273015415736849,
               'min_child_weight': 2, 'gamma': 0.6038476986548106, 'subsample': 0.7316641746804098,
               'colsample_bytree': 0.8006388061568654, 'scale_pos_weight': 1.050569703507208,
               'reg_alpha': 0.1437966726288049, 'reg_lambda': 0.3236297624663411}

# Features selected by PLS-XGBoost
XGB_Selected_features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']


def calculate_metrics(y_true, y_pred, y_prob):
    """Calculate all evaluation metrics"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    ppv = precision_score(y_true, y_pred)
    npv = tn / (tn + fn)
    f1 = f1_score(y_true, y_pred)
    auroc = roc_auc_score(y_true, y_prob)
    return {
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'AUROC': auroc,
        'PPV': ppv,
        'NPV': npv,
        'F1 Score': f1
    }


def preprocess_data(X_train, X_test):
    # Create imputer for missing values in numerical features
    num_imputer = SimpleImputer(strategy='median')
    # Create scaler for numerical features
    scaler = StandardScaler()

    # Get numerical and categorical columns
    numerical_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_columns = X_train.select_dtypes(include=['category']).columns

    # Make copies to avoid modifying original data
    X_train_processed = X_train.copy()
    X_test_processed = X_test.copy()

    # Process numerical features
    if len(numerical_columns) > 0:
        # Fit and transform on training data
        X_train_num = num_imputer.fit_transform(X_train_processed[numerical_columns])
        X_train_num = scaler.fit_transform(X_train_num)

        # Transform on test data
        X_test_num = num_imputer.transform(X_test_processed[numerical_columns])
        X_test_num = scaler.transform(X_test_num)

        # Convert back to DataFrame
        X_train_processed[numerical_columns] = X_train_num
        X_test_processed[numerical_columns] = X_test_num

    # Process categorical features
    if len(categorical_columns) > 0:
        # Fill missing values with mode for categorical columns
        for col in categorical_columns:
            mode_value = X_train_processed[col].mode()[0]
            X_train_processed[col] = X_train_processed[col].fillna(mode_value)
            X_test_processed[col] = X_test_processed[col].fillna(mode_value)

    return X_train_processed, X_test_processed


def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluate model performance on training and test sets"""
    # Evaluate on training set
    y_train_pred = model.predict(X_train)
    if isinstance(model, SVC):
        y_train_prob = model.decision_function(X_train)
        y_train_prob = 1 / (1 + np.exp(-y_train_prob))
    else:
        y_train_prob = model.predict_proba(X_train)[:, 1]
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_prob)

    # Evaluate on test set
    y_test_pred = model.predict(X_test)
    if isinstance(model, SVC):
        y_test_prob = model.decision_function(X_test)
        y_test_prob = 1 / (1 + np.exp(-y_test_prob))
    else:
        y_test_prob = model.predict_proba(X_test)[:, 1]
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_prob)

    return (train_metrics['Accuracy'], test_metrics['Accuracy'],
            train_metrics['Sensitivity'], test_metrics['Sensitivity'],
            train_metrics['Specificity'], test_metrics['Specificity'],
            train_metrics['AUROC'], test_metrics['AUROC'],
            train_metrics['PPV'], test_metrics['PPV'],
            train_metrics['NPV'], test_metrics['NPV'],
            train_metrics['F1 Score'], test_metrics['F1 Score'],
            y_test, y_test_pred,
            model)


def balance_samples(X, y, random_state=42):
    """Balance positive and negative samples"""
    # Convert y to DataFrame if needed
    if isinstance(y, pd.Series):
        y = pd.DataFrame(y)
    else:
        y = pd.DataFrame(y, columns=['target'])

    # Combine X and y
    X_y = pd.concat([X.reset_index(drop=True), y.reset_index(drop=True)], axis=1)

    # Get label column name
    y_col = y.columns[0]

    # Separate positive and negative samples
    negative_samples = X_y[X_y[y_col] == 0]
    positive_samples = X_y[X_y[y_col] == 1]

    # Get the number of minority class
    min_samples = min(len(negative_samples), len(positive_samples))

    # Downsample majority class
    if len(negative_samples) > min_samples:
        negative_samples = resample(negative_samples,
                                    n_samples=min_samples,
                                    random_state=random_state)
    if len(positive_samples) > min_samples:
        positive_samples = resample(positive_samples,
                                    n_samples=min_samples,
                                    random_state=random_state)

    # Combine and shuffle
    balanced_samples = pd.concat([negative_samples, positive_samples])
    balanced_samples = balanced_samples.sample(frac=1, random_state=random_state)

    # Split back into features and labels
    X_balanced = balanced_samples[X.columns]
    y_balanced = balanced_samples[y_col]

    return X_balanced, y_balanced


models = {
    'XGBoost': {
        'model': XGBClassifier(
            n_estimators=255,
            max_depth=10,
            learning_rate=0.09273015415736849,
            min_child_weight=2,
            gamma=0.6038476986548106,
            subsample=0.7316641746804098,
            colsample_bytree=0.8006388061568654,
            scale_pos_weight=1.050569703507208,
            reg_alpha=0.1437966726288049,
            reg_lambda=0.3236297624663411,
            random_state=42,
            enable_categorical=True
        ),
        'features': ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    },
    'LightGBM': {
        'model': LGBMClassifier(
            learning_rate=0.042628740412729516,
            n_estimators=163,
            num_leaves=46,
            max_depth=5,
            min_child_samples=15,
            feature_fraction=0.8611486887182361,
            bagging_fraction=0.8397657244664654,
            random_state=42
        ),
        'features': ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(
            n_estimators=100,
            learning_rate=0.05,
            random_state=42
        ),
        'features': ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    },
    'RandomForest': {
        'model': RandomForestClassifier(
            n_estimators=186,
            max_depth=10,
            min_samples_split=9,
            min_samples_leaf=3,
            max_features='log2',
            random_state=42
        ),
        'features': ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    },
    'SVM': {
        'model': SVC(
            C=0.1,
            gamma='scale',
            kernel='rbf',
            probability=True,
            random_state=42,
            cache_size=1000
        ),
        'features': ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    },
    'LogisticRegression': {
        'model': LogisticRegression(
            C=0.1,
            penalty='l2',
            solver='saga',
            max_iter=500,
            random_state=42
        ),
        'features': ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']
    }
}


def compute_mean_and_ci(results, idx):
    """Compute mean and 95% confidence interval"""
    values = [result[idx] for result in results]
    mean = np.mean(values)
    if len(values) < 2:
        return mean, 0
    try:
        sem = stats.sem(values)
        confidence_interval = stats.t.interval(confidence=0.95,
                                               df=len(values)-1,
                                               loc=mean,
                                               scale=sem)
        ci = confidence_interval[1] - mean
    except:
        ci = 0
    return mean, ci


# Dictionary to store evaluation metrics for all models
all_models_metrics = {model_name: {
    'final_fold_results': [],
    'y_test_all': [],
    'final_y_pred_prob': []
} for model_name in models.keys()}


# Train and evaluate each model
for model_name, model_info in models.items():
    print(f"\nTraining {model_name}...")

    if model_name == 'XGBoost':
        model = XGBClassifier(
            n_estimators=255,
            max_depth=10,
            learning_rate=0.09273015415736849,
            min_child_weight=2,
            gamma=0.6038476986548106,
            subsample=0.7316641746804098,
            colsample_bytree=0.8006388061568654,
            scale_pos_weight=1.050569703507208,
            reg_alpha=0.1437966726288049,
            reg_lambda=0.3236297624663411,
            random_state=42,
            enable_categorical=True
        )
    elif model_name == 'LightGBM':
        model = LGBMClassifier(
            learning_rate=0.042628740412729516,
            n_estimators=163,
            num_leaves=46,
            max_depth=5,
            min_child_samples=15,
            feature_fraction=0.8611486887182361,
            bagging_fraction=0.8397657244664654,
            random_state=42
        )
    elif model_name == 'RandomForest':
        model = RandomForestClassifier(
            n_estimators=186,
            max_depth=10,
            min_samples_split=9,
            min_samples_leaf=3,
            max_features='log2',
            random_state=42
        )
    elif model_name == 'SVM':
        model = SVC(
            C=0.1,
            gamma='scale',
            kernel='rbf',
            probability=True,
            random_state=42,
            cache_size=1000
        )
    elif model_name == 'AdaBoost':
        model = AdaBoostClassifier(
            n_estimators=100,
            learning_rate=0.05,
            random_state=42
        )
    elif model_name





### Test set-ROC curve of six machine learning models

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from scipy import stats

def plot_roc_curves(all_models_metrics):
    """Plot ROC curves for all models with mean AUC and standard deviation"""

    plt.figure(figsize=(8, 6), dpi=300)

    # Define colors for each model
    colors = {
        'XGBoost': '#4B7BE5',
        'LightGBM': '#2E8B57',
        'AdaBoost': '#A0522D',
        'RandomForest': '#DC3445',
        'SVM': '#FFA500',
        'LogisticRegression': '#9B51E0', 
    }

    # Set global font settings
    plt.rcParams.update({
        'font.family': 'Times New Roman',
        'font.serif': ['Times New Roman'],
        'font.size': 12,
        'axes.labelsize': 16, 
        'axes.titlesize': 16,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 12
    })

    # Plot ROC curve for each model
    for model_name, metrics in all_models_metrics.items():
        # Get true labels and predicted probabilities
        y_test = np.array(metrics['y_test_all'])
        y_pred_prob = np.array(metrics['final_y_pred_prob'])
        
        # Compute ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        
        # Calculate mean AUROC and its standard deviation across folds
        auroc_values = [result[7] for result in metrics['final_fold_results']]
        mean_auroc = np.mean(auroc_values)
        std_auroc = np.std(auroc_values)

        # Plot the ROC curve
        plt.plot(fpr, tpr, 
                 color=colors.get(model_name),  
                 linestyle='-',
                 lw=2,
                 label=f'{model_name} (AUC = {mean_auroc:.3f} ± {std_auroc:.3f})')
    
    # Plot diagonal line (no-skill classifier)
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1.5, alpha=0.7)
    
    # Customize plot
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('ROC Curves Comparison', fontsize=14, pad=20)
    
    # Add legend
    plt.legend(loc='lower right', fontsize=12, bbox_to_anchor=(1.0, 0.0),
               borderaxespad=1, framealpha=0.8, frameon=False)
    
    # Adjust layout
    plt.tight_layout()
    
    return plt.gcf()

# Example usage
fig = plot_roc_curves(all_models_metrics)
plt.show()

### Test set-Confusion matrix diagram of six machine learning models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(all_models_metrics):
    plt.rcParams.update({
        'font.family': 'Times New Roman',
        'font.serif': ['Times New Roman'],
        'font.size': 12,
        'axes.labelsize': 16,
        'axes.titlesize': 16,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 14
    })

    fig, axes = plt.subplots(2, 3, figsize=(15, 10), dpi=300)
    fig.suptitle('Average Confusion Matrices for Different Models\n(5-fold CV × 10 repeats)', 
                 fontsize=20, 
                 y=1.05, 
                 fontfamily='Times New Roman')

    cmap = plt.cm.Blues

    for idx, (model_name, metrics) in enumerate(all_models_metrics.items()):
        row = idx // 3
        col = idx % 3
        ax = axes[row, col]
        ax.set_facecolor('white')
        
        # 从metrics中获取数据
        y_true = np.array(metrics['y_test_all'])
        y_pred_prob = np.array(metrics['final_y_pred_prob'])
        
        # 分成50份（5折×10重复）
        n_splits = 50
        fold_size = len(y_true) // n_splits
        cms = []
        
        # 计算每一折的混淆矩阵
        for i in range(n_splits):
            start_idx = i * fold_size
            end_idx = (i + 1) * fold_size if i < n_splits-1 else len(y_true)
            fold_y_true = y_true[start_idx:end_idx]
            fold_y_pred = (y_pred_prob[start_idx:end_idx] > 0.5).astype(int)
            cm = confusion_matrix(fold_y_true, fold_y_pred)
            cms.append(cm)
        
        # 计算平均混淆矩阵
        cm = np.mean(cms, axis=0).round().astype(int)
        total = cm.sum()
        cm_percent = cm / total * 100
        cm_norm = cm.astype('float') / cm.max()
        
        im = ax.imshow(cm_norm, interpolation='nearest', 
                      cmap=cmap,
                      aspect='auto')
        
        for i in range(2):
            for j in range(2):
                color = "white" if cm_norm[i, j] > 0.7 else "black"
                # 显示平均值
                ax.text(j, i, str(cm[i, j]),
                       ha="center", va="center",
                       color=color,
                       fontsize=14,
                       fontfamily='Times New Roman',
                       fontweight='bold')
                # 显示百分比
                ax.text(j, i + 0.25,
                       f'({cm_percent[i,j]:.1f}%)',
                       ha='center', 
                       va='center',
                       color=color,
                       fontsize=12,
                       fontfamily='Times New Roman')
        
        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])
        ax.set_xticklabels(['Predicted\nNegative', 'Predicted\nPositive'],
                          fontfamily='Times New Roman', 
                          fontsize=14)
        ax.set_yticklabels(['Actual\nNegative', 'Actual\nPositive'],
                          fontfamily='Times New Roman', 
                          fontsize=14)
        
        ax.set_title(f'{model_name}', 
                    pad=10, 
                    fontsize=16, 
                    fontfamily='Times New Roman')
        
        for spine in ax.spines.values():
            spine.set_linewidth(1.0)
            
        ax.set_xticks(np.arange(-0.5, 2, 1), minor=True)
        ax.set_yticks(np.arange(-0.5, 2, 1), minor=True)
        ax.grid(which="minor", color="black", linestyle='-', linewidth=1)
        ax.tick_params(which="minor", size=0)

    fig.patch.set_facecolor('white')
    plt.tight_layout()
    plt.savefig('confusion_matrices_50_folds_average.png', dpi=300, bbox_inches='tight')
    
    return plt

# 使用示例:
plt = plot_confusion_matrices(all_models_metrics)
plt.show()


### Test set-DCA decision curve graph of six machine learning models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_net_benefit(y_true, y_pred_prob, threshold):
    """计算特定阈值下的Net Benefit"""
    y_pred = (y_pred_prob >= threshold).astype(int)
    TP = np.sum((y_pred == 1) & (y_true == 1))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    n = len(y_true)
    
    if TP + FP == 0:
        return 0
    else:
        return (TP/n - (FP/n) * (threshold/(1-threshold)))

def plot_dca_curves(all_models_metrics):
    # 创建图形和轴对象
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # 设置全局样式
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['axes.linewidth'] = 1.5
    plt.rcParams.update({
       'font.family': 'Times New Roman',
       'font.serif': ['Times New Roman'],
       'font.size': 12,
       'axes.labelsize': 16, 
       'axes.titlesize': 16,
       'xtick.labelsize': 16,
       'ytick.labelsize': 16,
       'legend.fontsize': 14
   })
    # 设置颜色映射
    colors = {
        'XGBoost': '#2E86C1',
        'LightGBM': '#27AE60',
        'RandomForest': '#8E44AD',
        'SVM': '#E67E22',
        'AdaBoost': '#C0392B',
        'LogisticRegression': '#34495E'
    }
    
    # 设置白色背景
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')
    
    # 定义阈值范围
    thresholds = np.arange(0.0, 1.01, 0.01)
    
    # 为每个模型计算和绘制DCA曲线
    for idx, (model_name, metrics) in enumerate(all_models_metrics.items()):
        y_true = np.array(metrics['y_test_all'])
        y_pred_prob = np.array(metrics['final_y_pred_prob'])
        
        # 计算net benefits
        net_benefits = [calculate_net_benefit(y_true, y_pred_prob, threshold) 
                       for threshold in thresholds]
        
        # 绘制DCA曲线
        ax.plot(thresholds, net_benefits, 
                color=list(colors.values())[idx],
                linewidth=2, 
                label=model_name)
    
    # 添加treat-all line
    treat_all_net_benefits = [calculate_net_benefit(y_true, 
                                                  np.ones_like(y_true), 
                                                  threshold)
                             for threshold in thresholds]
    ax.plot(thresholds, treat_all_net_benefits, color='black',linestyle='--', linewidth=1.5, label='Treat All')
    
    # 添加treat-none line
    ax.plot([0, 1], [0, 0], color='gray',linestyle='--', linewidth=1.5, label='Treat None')
    
    # 设置坐标轴范围和标签
    ax.set_xlim([0, 1])
    ax.set_ylim([-0.05, 0.6])  # 调整y轴范围
    ax.set_xlabel('Threshold Probability', fontsize=16)
    ax.set_ylabel('Net Benefit', fontsize=16)
    ax.set_title('Decision Curve Analysis', pad=15, fontsize=14)
    
    # 设置图例
    ax.legend(loc='upper right', 
              bbox_to_anchor=(1.0, 1.0),
              fontsize=12,
              frameon=False)
    
    # 设置刻度
    ax.tick_params(axis='both', which='major', labelsize=16)
    
    # 调整布局
    plt.tight_layout()
    
    # 保存图片
    plt.savefig('dca_curves_adjusted.png', dpi=300, bbox_inches='tight')
    
    return plt

# 使用示例：
plt = plot_dca_curves(all_models_metrics)
plt.show()

### Test set-SHAP summary plot for RF

In [ ]:
# 获取RF模型最后一个fold的训练结果
rf_results = all_models_metrics['RandomForest']['final_fold_results'][-1]
final_model = rf_results[-1]  # 获取模型

# 准备特征列表
features = ['age', 'pt', 'ptt', 'aniongap', 'bun', 'creatinine', 'wbc', 'mchc', 'rbc']

# 为了更好地显示，选择一部分样本
sample_size = 200
X_sample = X_test_processed[features].sample(n=min(sample_size, len(X_test_processed)), random_state=42)

# 设置matplotlib参数
plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 20,
    'axes.labelsize': 20,
    'axes.titlesize': 20,
    'xtick.labelsize': 26,
    'ytick.labelsize': 26
})

# 计算SHAP值
import shap
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_sample)

# 由于是二分类问题，我们选择正类的SHAP值
# 确保正确的维度
if isinstance(shap_values, list):
    print("Original SHAP values shapes:", [sv.shape for sv in shap_values])
    shap_values_plot = shap_values[1]  # 选择正类的SHAP值
    expected_value = explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value
else:
    print("Original SHAP values shape:", shap_values.shape)
    shap_values_plot = shap_values[:,:,1] if len(shap_values.shape) == 3 else shap_values
    expected_value = explainer.expected_value

print("Final SHAP values shape:", shap_values_plot.shape)
print("X_sample shape:", X_sample.shape)

# 绘制Summary Plot
plt.figure(figsize=(12, 10))
shap.summary_plot(
    shap_values_plot,
    X_sample,
    max_display=len(features),
    show=False,
    plot_size=None
)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# 获取当前轴对象
ax = plt.gca()
# 设置 x 轴和 y 轴刻度字体大小
ax.tick_params(axis='x', labelsize=20)  # x 轴刻度字体大小
ax.tick_params(axis='y', labelsize=20)  # y 轴刻度字体大小

# 设置 X 和 Y 轴标签
ax.set_xlabel("SHAP value (impact on model output)", fontsize=20, labelpad=20)  # 设置 X 轴标签
ax.set_ylabel("Features", fontsize=20, labelpad=20)  # 设置 Y 轴标签
# 直接通过 ax 设置标题
ax.set_title('Random Forest Model SHAP Summary Plot', fontsize=20, pad=20)
# 获取 colorbar（SHAP 颜色条）并调整字体大小
cbar = plt.gcf().axes[-1]  # 获取 colorbar 的 axis
cbar.tick_params(labelsize=20)  # 设置 colorbar 刻度字体大小
cbar.set_ylabel("Feature Value", fontsize=24)  # 设置 colorbar 标签字体大小

# 调整 colorbar 上 "Low" 和 "High" 文字的字体大小
for text in cbar.get_yticklabels():  
    text.set_fontsize(20)  # 设置 colorbar 刻度字体大小

#plt.title('Random Forest Model SHAP Summary Plot', fontsize=24,pad=20)
plt.tight_layout()
plt.show()


### Test set-SHAP bar plot for RF

In [ ]:
# 计算每个特征的平均绝对SHAP值
import numpy as np
import pandas as pd

# 计算每个特征的平均绝对SHAP值
mean_shap_values = np.abs(shap_values_plot).mean(axis=0)

# 将特征和对应的平均SHAP值存入DataFrame中
feature_importance_df = pd.DataFrame({
    'Feature': X_sample.columns,
    'Mean SHAP Value': mean_shap_values
})

# 按照Mean SHAP值排序
feature_importance_df = feature_importance_df.sort_values(by='Mean SHAP Value', ascending=True)

# 绘制条形图
plt.figure(figsize=(12, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Mean SHAP Value'], color='blue')
plt.xlabel('Mean Absolute SHAP Value', fontsize=20)
plt.ylabel('Features', fontsize=20)
#plt.title('Feature Importance based on Mean Absolute SHAP Value', fontsize=24)

# 设置轴标签的字体大小
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

# 显示图形
plt.tight_layout()
plt.show()
